<h3> Load Libraries </h3>

In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.externals import joblib
import matplotlib.pyplot as plt
import os
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
from time import sleep
from random import randint
import glob
import json
import zipfile, io

from google.cloud import storage
from zipfile import ZipFile
from zipfile import is_zipfile
import io

import rasterio as rio
from rasterio.plot import plotting_extent
from matplotlib.patches import Patch
from matplotlib.colors import ListedColormap, BoundaryNorm
import earthpy as et
import pyproj
from requests_respectful import RespectfulRequester
from ratelimit import limits, RateLimitException, sleep_and_retry
from backoff import on_exception, expo
from google.cloud import storage
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from pyspark.serializers import MarshalSerializer
from os import listdir
from os.path import isfile, join
from tenacity import *
from multiprocessing import Pool
import wget
import clint
from gcsfs import GCSFileSystem

In [2]:
b

<h3> Global Variables </h3>

In [3]:
BUCKET_NAME = "agriculture-bucket-wgr"
base_path = "gs://" + BUCKET_NAME  + "/Data sets/"
site_id_path = "US-Wgr"
site_id = "US-Wgr"
site_path = os.path.join(base_path, site_id_path + "/")
print(site_path)
landsat_url = "https://eeflux-level1.appspot.com/landsat"
landsat_selector_url = "https://eeflux-level1.appspot.com/select_landsat"
headers = {
    'Content-Type': 'text/html; charset=utf-8',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:50.0) Gecko/20100101 Firefox/50.0',
    'referrer': 'https://google.com'
}

MINUTES_MAX = 600
year = "_EEflux"
type_id = 1
from_to = "2004-01-01 to 2011-12-31"

gs://agriculture-bucket-wgr/Data sets/US-Wgr/
gs://agriculture-bucket-wgr/Data sets/US-Wgr/


<h3> Helpers </h3>

In [4]:
def export_csv(df, fileName):
    export_path = os.path.join(base_path, fileName + ".csv")
    export_csv = df.to_csv (export_path,
                        index = None, header=True)
def load_data(fileName):
    file_path = os.path.join(base_path, fileName + ".csv")
    df = pd.read_csv(file_path, delimiter=',')
    return df

In [5]:
file_path = os.path.join(base_path, "filtered_sites.xlsx")
sites_df = pd.read_excel(file_path)
sites_df["Start Year"] = ""
sites_df["End Year"] = ""
sites_df["date_info"] = ""
sites_df.head()

,Site Id,Latitude,Longitude,Years Data Collected:,Start Year,End Year,date_info
0,US-ARM,36.605800,-97.488800,2002 - 2019,,,
1,US-AR2,36.635800,-99.597500,2009 - 2012,,,
2,US-KFS,39.056100,-95.190700,2007 - 2019,,,
3,US-Wgr,45.112865,-122.656026,2014 - 2019,,,
4,US-Kon,39.082400,-96.560300,2006 - 2019,,,


,Site Id,Latitude,Longitude,Years Data Collected:,Start Year,End Year,date_info
0,US-ARM,36.605800,-97.488800,2002 - 2019,,,
1,US-AR2,36.635800,-99.597500,2009 - 2012,,,
2,US-KFS,39.056100,-95.190700,2007 - 2019,,,
3,US-Wgr,45.112865,-122.656026,2014 - 2019,,,
4,US-Kon,39.082400,-96.560300,2006 - 2019,,,


<h4> 3. Separate year into from - to intervals and add a column for date info </h4>

In [6]:
for i in range(len(sites_df)):
    data = sites_df.iloc[i,3].split("-")
    sites_df.iloc[i, 4] = data[0].strip(' \t\n\r')
    sites_df.iloc[i, 5] = "2019"
    sites_df.iloc[i, 6] = sites_df.iloc[i, 4] + "-01-01 to " + sites_df.iloc[i, 5] + "-12-30"
#     sites_df.iloc[i, 6] = from_to
    
#TODO: check which day, month to use for each from and to year per site

<h4> 4. Get list of date images per site </h4>

In [7]:
#Pick a site_id and gets its information
def get_site_info(site_id):
    site_cond = sites_df['Site Id'] == site_id
    return sites_df[site_cond]

site_df = get_site_info(site_id)
site_df

,Site Id,Latitude,Longitude,Years Data Collected:,Start Year,End Year,date_info
3,US-Wgr,45.112865,-122.656026,2014 - 2019,2014,2019,2014-01-01 to 2019-12-30


,Site Id,Latitude,Longitude,Years Data Collected:,Start Year,End Year,date_info
3,US-Wgr,45.112865,-122.656026,2014 - 2019,2014,2019,2014-01-01 to 2019-12-30


<h4> 5. Retrieve only Latitude, Longitude, Start Year, & End Year for parameter request </h4>

In [8]:
#Create request parameters
def get_request_param(site_df):
    site_info = site_df.loc[:, ["Latitude", "Longitude", "date_info"]]
    site_info.rename(columns={'Latitude': 'lat', 'Longitude' : 'lng'}, inplace=True)
    return site_info

param = get_request_param(site_df)
param

# param["date_info"] = from_to
# param

,lat,lng,date_info
3,45.112865,-122.656026,2014-01-01 to 2019-12-30


,lat,lng,date_info
3,45.112865,-122.656026,2014-01-01 to 2019-12-30


<h4> 6. Post request to get image information for each date per site </h4>

In [9]:
def to_list_dict(param):
    '''Converts an object to a list of dictionary'''
    #Convert python object to list
    param = param.to_dict(orient='records')
    #Take only the first child of the list
    return param[0]

def clean_param(param):
    '''Fixes the format for latitude & longitude'''
    param["lat"] = "" + str(param["lat"]) + ""
    param["lng"] = "" + str(param["lng"]) + ""
    return param

def post_request(url, param):
    '''Issues a post request'''
    response = requests.post(url, headers=headers, data=json.dumps(param))
    response.status_code
    return response

def parse_html(response):
    '''Parses the content of the response'''
    doc = BeautifulSoup(response.text, 'html.parser')
    return str(doc)

def process_response(url, param):
    '''Process the contents of the page and returns its response along with the
    cleaned parameters'''
    param = to_list_dict(param)
    param = clean_param(param)
    print("url:", url)
    print("params:", param)
    response = post_request(url, param)
    return (param, parse_html(response))

param, response = process_response(landsat_url, param)
print(response)

url: https://eeflux-level1.appspot.com/landsat
params: {'lat': '45.112865', 'lng': '-122.656026', 'date_info': '2014-01-01 to 2019-12-30'}
url: https://eeflux-level1.appspot.com/landsat
params: {'lat': '45.112865', 'lng': '-122.656026', 'date_info': '2014-01-01 to 2019-12-30'}
{"0": {"date": "2014-01-03", "tier": "T1", "id": "LE70460292014003EDC00", "cloud": 32.0}, "1": {"date": "2014-01-19", "tier": "T1", "id": "LE70460292014019EDC00", "cloud": 51.0}, "2": {"date": "2014-03-24", "tier": "T1", "id": "LE70460292014083EDC00", "cloud": 22.0}, "3": {"date": "2014-04-09", "tier": "T1", "id": "LE70460292014099EDC00", "cloud": 21.0}, "4": {"date": "2014-05-11", "tier": "T1", "id": "LE70460292014131EDC00", "cloud": 38.0}, "5": {"date": "2014-07-14", "tier": "T1", "id": "LE70460292014195EDC00", "cloud": 16.0}, "6": {"date": "2014-07-30", "tier": "T1", "id": "LE70460292014211EDC00", "cloud": 3.0}, "7": {"date": "2014-08-31", "tier": "T1", "id": "LE70460292014243EDC00", "cloud": 40.0}, "8": {"dat

{"0": {"date": "2014-01-03", "tier": "T1", "id": "LE70460292014003EDC00", "cloud": 32.0}, "1": {"date": "2014-01-19", "tier": "T1", "id": "LE70460292014019EDC00", "cloud": 51.0}, "2": {"date": "2014-03-24", "tier": "T1", "id": "LE70460292014083EDC00", "cloud": 22.0}, "3": {"date": "2014-04-09", "tier": "T1", "id": "LE70460292014099EDC00", "cloud": 21.0}, "4": {"date": "2014-05-11", "tier": "T1", "id": "LE70460292014131EDC00", "cloud": 38.0}, "5": {"date": "2014-07-14", "tier": "T1", "id": "LE70460292014195EDC00", "cloud": 16.0}, "6": {"date": "2014-07-30", "tier": "T1", "id": "LE70460292014211EDC00", "cloud": 3.0}, "7": {"date": "2014-08-31", "tier": "T1", "id": "LE70460292014243EDC00", "cloud": 40.0}, "8": {"date": "2014-09-16", "tier": "T1", "id": "LE70460292014259EDC00", "cloud": 17.0}, "9": {"date": "2014-10-02", "tier": "T1", "id": "LE70460292014275EDC00", "cloud": 0.0}, "10": {"date": "2014-10-18", "tier": "T1", "id": "LE70460292014291EDC00", "cloud": 62.0}, "11": {"date": "2015-

In [10]:
print("Site " + site_id + " has " + str(len(response)) + " rasters")

Site US-Wgr has 23718 rasters
Site US-Wgr has 23718 rasters


<h4> 7. Convert Json to python object & Export all information </h4>

In [11]:
def to_dict(response):
    '''Converts an object to a dictionary'''
    return json.loads(response)

data = to_dict(response)
print(data)
print(len(data))

{'0': {'date': '2014-01-03', 'tier': 'T1', 'id': 'LE70460292014003EDC00', 'cloud': 32.0}, '1': {'date': '2014-01-19', 'tier': 'T1', 'id': 'LE70460292014019EDC00', 'cloud': 51.0}, '2': {'date': '2014-03-24', 'tier': 'T1', 'id': 'LE70460292014083EDC00', 'cloud': 22.0}, '3': {'date': '2014-04-09', 'tier': 'T1', 'id': 'LE70460292014099EDC00', 'cloud': 21.0}, '4': {'date': '2014-05-11', 'tier': 'T1', 'id': 'LE70460292014131EDC00', 'cloud': 38.0}, '5': {'date': '2014-07-14', 'tier': 'T1', 'id': 'LE70460292014195EDC00', 'cloud': 16.0}, '6': {'date': '2014-07-30', 'tier': 'T1', 'id': 'LE70460292014211EDC00', 'cloud': 3.0}, '7': {'date': '2014-08-31', 'tier': 'T1', 'id': 'LE70460292014243EDC00', 'cloud': 40.0}, '8': {'date': '2014-09-16', 'tier': 'T1', 'id': 'LE70460292014259EDC00', 'cloud': 17.0}, '9': {'date': '2014-10-02', 'tier': 'T1', 'id': 'LE70460292014275EDC00', 'cloud': 0.0}, '10': {'date': '2014-10-18', 'tier': 'T1', 'id': 'LE70460292014291EDC00', 'cloud': 62.0}, '11': {'date': '2015-

{'0': {'date': '2014-01-03', 'tier': 'T1', 'id': 'LE70460292014003EDC00', 'cloud': 32.0}, '1': {'date': '2014-01-19', 'tier': 'T1', 'id': 'LE70460292014019EDC00', 'cloud': 51.0}, '2': {'date': '2014-03-24', 'tier': 'T1', 'id': 'LE70460292014083EDC00', 'cloud': 22.0}, '3': {'date': '2014-04-09', 'tier': 'T1', 'id': 'LE70460292014099EDC00', 'cloud': 21.0}, '4': {'date': '2014-05-11', 'tier': 'T1', 'id': 'LE70460292014131EDC00', 'cloud': 38.0}, '5': {'date': '2014-07-14', 'tier': 'T1', 'id': 'LE70460292014195EDC00', 'cloud': 16.0}, '6': {'date': '2014-07-30', 'tier': 'T1', 'id': 'LE70460292014211EDC00', 'cloud': 3.0}, '7': {'date': '2014-08-31', 'tier': 'T1', 'id': 'LE70460292014243EDC00', 'cloud': 40.0}, '8': {'date': '2014-09-16', 'tier': 'T1', 'id': 'LE70460292014259EDC00', 'cloud': 17.0}, '9': {'date': '2014-10-02', 'tier': 'T1', 'id': 'LE70460292014275EDC00', 'cloud': 0.0}, '10': {'date': '2014-10-18', 'tier': 'T1', 'id': 'LE70460292014291EDC00', 'cloud': 62.0}, '11': {'date': '2015-

<h4> 8. Export raster information per site </h4>

In [12]:
def get_site_rasters(data):
    '''Gets a site's information from EEflux (date, image, cloud, tier)'''
    images = []
    dates = []
    clouds = []
    tiers = []
    for i in range(len(data)):
        images.append(data[str(i)]["id"])
        dates.append(data[str(i)]["date"])
        clouds.append(data[str(i)]["cloud"])
        tiers.append(data[str(i)]["tier"])
    
    df = pd.DataFrame({'Date': dates, 'Tier': tiers,'Image Id': images, 'Cloud': clouds})
#     df["Date"] = pd.to_datetime(df["Date"], format="%Y-%m-%d")
    df.drop_duplicates(keep="first",inplace=True) 
    df.sort_values(by=["Date"], inplace=True)
    return df

site_images_date_df = get_site_rasters(data)
site_images_date_df[get_column_name(type_id)] = ""
# site_images_date_df["Mean Modeled ET"] = ""
site_images_date_df.head(30)


export_csv(site_images_date_df, site_id + year)

<h4> 9. Specify image filter information to download </h4>

In [13]:
def get_url_suffix(type_id):
    """Gets the url suffix depending on the type of the specified filter"""
    url_suffix = ""
    if type_id == 1:
        url_suffix = "download_eta"
    elif type_id == 2:
        url_suffix = "download_etr"
    elif type_id == 3:
        url_suffix = "download_eto"
    elif type_id == 4:
        url_suffix = "download_LST"
    return url_suffix

def get_url_filter_parm(type_id):
    url_param = ""
    filter_name = ""
    if type_id == 1:
        url_param = "eta"
        filter_name = "_ETA"
    elif type_id == 2:
        url_param = "eta"
        filter_name = "_ETA"
    elif type_id == 3:
        url_param = "eta"
        filter_name = "_ETA"
    elif type_id == 4:
        url_param = "eta"
        filter_name = "_ETA"
    return (url_param, filter_name)

def get_column_name(type_id):
    column_name = ""
    if type_id == 1:
        column_name = "Modeled ET"
    elif type_id == 2:
        column_name = "Modeled ET"
    elif type_id == 3:
        column_name = "Modeled ET"
    elif type_id == 4:
        column_name = "Modeled ET"
    
        
def get_image_filter_params(type_id, param):
    """Gets the parameters for the url depending on the image filter"""
    url_suffix = get_url_suffix(type_id)
    url = "https://eeflux-level1.appspot.com/" + url_suffix
    response = post_request(url, param)
    return response

def get_image_filters(param, type_id):
    #Get information for a metric depending on the type:
    params = {}
    response = get_image_filter_params(type_id, param)
    if response.status_code == 200:
        params = parse_html(response)
        print("params:",params)
    return params


<h3> 11. Get metric value from raster </h3>

In [31]:
def get_data_from_image(path, latitude, longitude, metric_name):
    '''Retrieves a metric value based on the supplied latitude and longitude'''
    fs = GCSFileSystem()
    fp = fs.open(path)
    with rio.open(fp) as src:
        im = src.read(1, masked=True)
        spatial_extent = plotting_extent(src)
        data = src.read(1)
        # Use pyproj to convert point coordinates
        utm = pyproj.Proj(src.crs) # Pass CRS of image from rasterio
        lonlat = pyproj.Proj(init='epsg:4326')

        lon,lat = (longitude, latitude)
        east,north = pyproj.transform(lonlat, utm, lon, lat)

        print(metric_name + '\n-------')
        print(f'Longitude,Latitude=\t\t({lon:.2f},{lat:.2f})')
        print(f'East,North=\t({east:g},{north:g})')

        row, col = src.index(east, north) # spatial --> image coordinates
        print(f'Row,Col=\t\t({row},{col})')
        
        val, neighbors_avg_val = get_neighbors(data, row, col)
        print("Average value of pixels:", neighbors_avg_val)
        
        return val, neighbors_avg_val


#Assuming we have 8 neighbors and all are 9
#The site coordinate is in the center
def get_neighbors(data, row, column):
    c_row = row
    c_column = column
    
    f_row = c_row - 1
    l_row = c_row + 1
    
    f_column = c_column - 1
    l_column = c_column + 1
    
    site_value = data[c_row, c_column]
    
    values = [data[f_row, f_column], data[f_row, c_column], data[f_row, l_column],
              data[c_row, f_row], site_value, data[c_row, l_row],
              data[l_row, f_column],data[l_row, c_column], data[l_row, l_column]]
    
    print("values of all pixels:", values)
    pixels_avg = np.mean(values)
    
    return site_value, pixels_avg
                  
def zipextract(bucketname, zipfilename_with_path, file_name, column_name):
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucketname)

    destination_blob_pathname = zipfilename_with_path
    blob = bucket.blob(destination_blob_pathname)
    zipbytes = io.BytesIO(blob.download_as_string())
    folder_extracted_name = zipfilename_with_path.split(".")[0]

    if is_zipfile(zipbytes):
        with ZipFile(zipbytes, 'r') as myzip:
            for contentfilename in myzip.namelist():
                contentfile = myzip.read(contentfilename)
                blob = bucket.blob(folder_extracted_name + "/" + contentfilename)
                blob.upload_from_string(contentfile)
    
                if contentfilename.endswith(".tif"):
                    row = file_name.split("_")
                    date = row[0]
                    image_id = row[1]
                    metric_name = row[2].split(".")[0]
                    print("date:", date, "image:", image_id, "metric:", metric_name)

                    raster_path = "gs://" + bucketname + "/" + folder_extracted_name + "/" + contentfilename
                    metric_value, mean_value = get_data_from_image(raster_path, site_df.iat[0,1],
                                                                site_df.iat[0,2], metric_name)
                    print("data is", metric_value)

                    set_metric(site_images_date_df, image_id, metric_name, metric_value, mean_value, column_name)  

#                     blob.delete()

#     blob.delete()
             
        
def set_metric(site_eeflux_df, image_id, metric_name, metric_value, mean_value, column_name):
    site_images_date_df = load_data(site_id + year)
    site_images_date_df.loc[site_images_date_df['Image Id'] == image_id, column_name] = metric_value
#     site_images_date_df.loc[site_images_date_df['Image Id'] == image_id, "Mean Modeled ET"] = mean_value
    print(site_images_date_df.loc[site_images_date_df['Image Id'] == image_id,:])
    export_csv(site_images_date_df, site_id + year)
    
    
def unescape(s):
    """Remove un-needed characters from url"""
    s = s.replace("&lt;", "<")
    s = s.replace("&gt;", ">")
    s = s.replace("&amp;", "&")
    s = s.replace(";", "")
    return s

def upload_files(bucketName, localFolder, file_name):
    """Upload files to GCP bucket."""
    localFile = "/Data sets/" + site_id_path + "/" + file_name
    fullPath = os.path.join(site_path, file_name)
    
    client = storage.Client()
    bucket = client.get_bucket(bucketName)
    blob = bucket.blob(localFile)
    blob.upload_from_filename(fullPath)
    print("upload traa")
    
    return f'Uploaded {file_name} to "{bucketName}" bucket.'


# @on_exception(expo, RateLimitException, max_tries=10)
# @sleep_and_retry
# @limits(calls=20, period=MINUTES_MAX)
# @retry(stop=stop_after_attempt(10))
def download_image(image_id, type_id):
    """Download an image depending on provided filter of ET"""
    param_image = param.copy()  
    param_image.update( {'image_id' : image_id} )
    filter_dict = get_image_filters(param_image, type_id)
    if filter_dict:
        (url_param, filter_name) = get_url_filter_parm(type_id)
        url = json.loads(filter_dict)[url_param]["url"]
        url = unescape(url)
    headers = {
            'Content-Type': 'text/html; charset=utf-8',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.5',
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:50.0) Gecko/20100101 Firefox/50.0',
            'referrer': 'https://google.com',
            'Accept-Encoding': 'gzip, deflate',
            'Pragma': 'no-cache'
        }
    print("url:", url)
    try:
        response = requests.get(url, headers=headers, allow_redirects=True)
        print("response download:", response)

        if response.status_code == 200:
            selected_image_object = site_images_date_df[site_images_date_df["Image Id"] == image_id]
            file_name = str(selected_image_object.iloc[0,0]) + "_" + image_id + filter_name + ".zip"
            file_path = os.path.join(site_path, file_name)

            sub_directory = "Data sets/" + site_id_path + "/"
            blob_file_path = sub_directory + file_name
            upload_file_path = file_path

            print("blob file path:", blob_file_path)
            print("upload path:", upload_file_path)

            client = storage.Client()
            bucket = client.get_bucket(BUCKET_NAME)
            blob = bucket.blob(blob_file_path)

            filename = wget.download(url)
            
            blob.upload_from_filename(filename)
            
            zipextract(BUCKET_NAME, blob_file_path, file_name, get_column_name(type))


        elif response.status_code == 429:
            print("Failed at image id:", image_id)
            failed_images.append(image_id)
        else:
            print("Response error:", response.status_code)
    except Exception as ex:
        print("Exception:",str(ex))

<h3> 12. Run the procedure using parallel threads </h3>

In [ ]:
start_time = time.time()
images = []
for i in range(site_images_date_df.shape[0]):
    selected_object = site_images_date_df.iloc[i,:]
    selected_image_id = selected_object["Image Id"]
    images.append(selected_image_id)
    
for i in range(len(images)):   
    download_image(images[i], type_id)
    
elapsed_time = time.time() - start_time
time.strftime("%H:%M:%S", time.gmtime(elapsed_time))

eta: {"eta": {"url": "https://earthengine.googleapis.com/api/download?docid=568ea4177224e9764b3ac2fd283cd2ce&amp;token;=e38f1d93df63ef0dbfaadecf7a66fc95"}}
url: https://earthengine.googleapis.com/api/download?docid=568ea4177224e9764b3ac2fd283cd2ce&token=e38f1d93df63ef0dbfaadecf7a66fc95
eta: {"eta": {"url": "https://earthengine.googleapis.com/api/download?docid=568ea4177224e9764b3ac2fd283cd2ce&amp;token;=e38f1d93df63ef0dbfaadecf7a66fc95"}}
url: https://earthengine.googleapis.com/api/download?docid=568ea4177224e9764b3ac2fd283cd2ce&token=e38f1d93df63ef0dbfaadecf7a66fc95
response download: <Response [200]>
file path gs://agriculture-bucket-wgr/Data sets/US-Wgr/2014-01-03_LE70460292014003EDC00_ETA.zip
blob file path: Data sets/US-Wgr/2014-01-03_LE70460292014003EDC00_ETA.zip
upload path: gs://agriculture-bucket-wgr/Data sets/US-Wgr/2014-01-03_LE70460292014003EDC00_ETA.zip
response download: <Response [200]>
file path gs://agriculture-bucket-wgr/Data sets/US-Wgr/2014-01-03_LE70460292014003EDC

         Date Tier               Image Id  Cloud  Modeled ET  Mean Modeled ET
3  2014-01-27   T2  LC80460292014027LGN01  100.0     0.43219         0.338347
content file: LC80460292014027LGN01_ETa.eta.tif
date: 2014-01-27 image: LC80460292014027LGN01 metric: ETA
ETA
-------
Longitude,Latitude=		(-122.66,45.11)
East,North=	(527057,4.99555e+06)
Row,Col=		(2062,4579)
values of all pixels: [0.43562028, 0.4434961, 0.4469839, 0.0, 0.43218952, 0.0, 0.42073295, 0.42905056, 0.43705198]
Average value of pixels: 0.3383473
data is 0.43218952
         Date Tier               Image Id  Cloud  Modeled ET  Mean Modeled ET
3  2014-01-27   T2  LC80460292014027LGN01  100.0     0.43219         0.338347
eta: {"eta": {"url": "https://earthengine.googleapis.com/api/download?docid=7f073c0c466335cf3c2391184562f88c&amp;token;=5a1bc63babde5baf512ca524fcddd001"}}
url: https://earthengine.googleapis.com/api/download?docid=7f073c0c466335cf3c2391184562f88c&token=5a1bc63babde5baf512ca524fcddd001
eta: {"eta": {"url": "

eta: {"eta": {"url": "https://earthengine.googleapis.com/api/download?docid=a05bc4dec06745d0ebfb1d2b308f24bc&amp;token;=6c1e3bbd5c075c50678ad58c9adafe9b"}}
url: https://earthengine.googleapis.com/api/download?docid=a05bc4dec06745d0ebfb1d2b308f24bc&token=6c1e3bbd5c075c50678ad58c9adafe9b
eta: {"eta": {"url": "https://earthengine.googleapis.com/api/download?docid=a05bc4dec06745d0ebfb1d2b308f24bc&amp;token;=6c1e3bbd5c075c50678ad58c9adafe9b"}}
url: https://earthengine.googleapis.com/api/download?docid=a05bc4dec06745d0ebfb1d2b308f24bc&token=6c1e3bbd5c075c50678ad58c9adafe9b
response download: <Response [200]>
file path gs://agriculture-bucket-wgr/Data sets/US-Wgr/2014-02-28_LC80460292014059LGN01_ETA.zip
blob file path: Data sets/US-Wgr/2014-02-28_LC80460292014059LGN01_ETA.zip
upload path: gs://agriculture-bucket-wgr/Data sets/US-Wgr/2014-02-28_LC80460292014059LGN01_ETA.zip
response download: <Response [200]>
file path gs://agriculture-bucket-wgr/Data sets/US-Wgr/2014-02-28_LC80460292014059LGN

         Date Tier               Image Id  Cloud  Modeled ET  Mean Modeled ET
7  2014-03-08   T2  LE70460292014067EDC00  100.0         0.0              0.0
         Date Tier               Image Id  Cloud  Modeled ET  Mean Modeled ET
7  2014-03-08   T2  LE70460292014067EDC00  100.0         0.0              0.0
eta: {"eta": {"url": "https://earthengine.googleapis.com/api/download?docid=bf43b59c4d55ac8a7daa875f5d604605&amp;token;=d22d60403cb9bff62c0c2b323cd36638"}}
url: https://earthengine.googleapis.com/api/download?docid=bf43b59c4d55ac8a7daa875f5d604605&token=d22d60403cb9bff62c0c2b323cd36638
eta: {"eta": {"url": "https://earthengine.googleapis.com/api/download?docid=bf43b59c4d55ac8a7daa875f5d604605&amp;token;=d22d60403cb9bff62c0c2b323cd36638"}}
url: https://earthengine.googleapis.com/api/download?docid=bf43b59c4d55ac8a7daa875f5d604605&token=d22d60403cb9bff62c0c2b323cd36638
eta: {"eta": {"url": "https://earthengine.googleapis.com/api/download?docid=bf43b59c4d55ac8a7daa875f5d604605&amp;t

content file: LE70460292014083EDC00_ETa.eta.tfw
content file: LE70460292014083EDC00_ETa.eta.tif
date: 2014-03-24 image: LE70460292014083EDC00 metric: ETA
ETA
-------
Longitude,Latitude=		(-122.66,45.11)
East,North=	(527057,4.99555e+06)
Row,Col=		(1782,4659)
values of all pixels: [2.9376683, 3.093035, 3.4070585, 0.0, 3.065965, 0.0, 2.952177, 3.0981383, 3.3847306]
Average value of pixels: 2.4376416
data is 3.065965
         Date Tier               Image Id  Cloud  Modeled ET  Mean Modeled ET
9  2014-03-24   T1  LE70460292014083EDC00   22.0    3.065965         2.437642
content file: LE70460292014083EDC00_ETa.eta.tfw
content file: LE70460292014083EDC00_ETa.eta.tif
date: 2014-03-24 image: LE70460292014083EDC00 metric: ETA
ETA
-------
Longitude,Latitude=		(-122.66,45.11)
East,North=	(527057,4.99555e+06)
Row,Col=		(1782,4659)
values of all pixels: [2.9376683, 3.093035, 3.4070585, 0.0, 3.065965, 0.0, 2.952177, 3.0981383, 3.3847306]
Average value of pixels: 2.4376416
data is 3.065965
         D

          Date Tier               Image Id  Cloud  Modeled ET  Mean Modeled ET
11  2014-04-09   T1  LE70460292014099EDC00   21.0    1.769592         2.062438
content file: LE70460292014099EDC00_ETa.eta.tfw
content file: LE70460292014099EDC00_ETa.eta.tif
date: 2014-04-09 image: LE70460292014099EDC00 metric: ETA
ETA
-------
Longitude,Latitude=		(-122.66,45.11)
East,North=	(527057,4.99555e+06)
Row,Col=		(1782,4639)
values of all pixels: [1.1926304, 1.7190869, 2.2414458, 2.938467, 1.769592, 2.9560728, 1.3895104, 2.000535, 2.3545983]
Average value of pixels: 2.0624378
data is 1.769592
          Date Tier               Image Id  Cloud  Modeled ET  Mean Modeled ET
11  2014-04-09   T1  LE70460292014099EDC00   21.0    1.769592         2.062438
content file: LE70460292014099EDC00_ETa.eta.tfw
content file: LE70460292014099EDC00_ETa.eta.tif
date: 2014-04-09 image: LE70460292014099EDC00 metric: ETA
ETA
-------
Longitude,Latitude=		(-122.66,45.11)
East,North=	(527057,4.99555e+06)
Row,Col=		(1782,463

date: 2014-04-17 image: LC80460292014107LGN01 metric: ETA
ETA
-------
Longitude,Latitude=		(-122.66,45.11)
East,North=	(527057,4.99555e+06)
Row,Col=		(2062,4559)
values of all pixels: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Average value of pixels: 0.0
data is 0.0
          Date Tier               Image Id  Cloud  Modeled ET  Mean Modeled ET
12  2014-04-17   T2  LC80460292014107LGN01  100.0         0.0              0.0
eta: {"eta": {"url": "https://earthengine.googleapis.com/api/download?docid=eb759db61cb793ab6cdda215db89481b&amp;token;=6835b94b070db66d6b40d540d5e8e787"}}
url: https://earthengine.googleapis.com/api/download?docid=eb759db61cb793ab6cdda215db89481b&token=6835b94b070db66d6b40d540d5e8e787
response download: <Response [200]>
file path gs://agriculture-bucket-wgr/Data sets/US-Wgr/2014-04-25_LE70460292014115EDC00_ETA.zip
blob file path: Data sets/US-Wgr/2014-04-25_LE70460292014115EDC00_ETA.zip
upload path: gs://agriculture-bucket-wgr/Data sets/US-Wgr/2014-04-25_LE7046029

response download: <Response [200]>
file path gs://agriculture-bucket-wgr/Data sets/US-Wgr/2014-05-11_LE70460292014131EDC00_ETA.zip
blob file path: Data sets/US-Wgr/2014-05-11_LE70460292014131EDC00_ETA.zip
upload path: gs://agriculture-bucket-wgr/Data sets/US-Wgr/2014-05-11_LE70460292014131EDC00_ETA.zip
response download: <Response [200]>
file path gs://agriculture-bucket-wgr/Data sets/US-Wgr/2014-05-11_LE70460292014131EDC00_ETA.zip
blob file path: Data sets/US-Wgr/2014-05-11_LE70460292014131EDC00_ETA.zip
upload path: gs://agriculture-bucket-wgr/Data sets/US-Wgr/2014-05-11_LE70460292014131EDC00_ETA.zip
response download: <Response [200]>
file path gs://agriculture-bucket-wgr/Data sets/US-Wgr/2014-05-11_LE70460292014131EDC00_ETA.zip
blob file path: Data sets/US-Wgr/2014-05-11_LE70460292014131EDC00_ETA.zip
upload path: gs://agriculture-bucket-wgr/Data sets/US-Wgr/2014-05-11_LE70460292014131EDC00_ETA.zip
response download: <Response [200]>
file path gs://agriculture-bucket-wgr/Data sets/U

In [91]:
def list_files(bucketName):
    """List all files in GCP bucket."""
    files = bucket.list_blobs()
    fileList = [file.name for file in files if '.' in file.name]
    return fileList

files = list_files(BUCKET_NAME)
files

['Data sets/US-Wgr/2014-01-03_LE70460292014003EDC00_ETA.zip',
 'Data sets/US-Wgr_EEflux.csv',
 'Data sets/US-Wgr_EEflux_x.csv',
 'Data sets/filtered_sites.xlsx',
 'google-cloud-dataproc-metainfo/ee25e645-aa98-4177-a99b-6f67984d80aa/cluster.properties',
 'notebooks/jupyter/.ipynb_checkpoints/EEFlux_Scrap_Test (1)-a6a0c9f3-879a-46bd-b6fb-53d8280bdc1a.ipynb',
 'notebooks/jupyter/.ipynb_checkpoints/EEFlux_Scrap_Test (1)-b3a91ac4-6d25-41c3-9731-e3b9712f499e.ipynb',
 'notebooks/jupyter/.ipynb_checkpoints/EEFlux_Scrap_Test (1)-b7d7e9e6-d7c1-41ae-9009-959ebba2dcc9.ipynb',
 'notebooks/jupyter/EEFlux_Scrap_Test (1).ipynb']

In [22]:
from google.cloud import storage
from zipfile import ZipFile
from zipfile import is_zipfile
import io

def zipextract(bucketname, zipfilename_with_path):

    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucketname)

    destination_blob_pathname = zipfilename_with_path

    blob = bucket.blob(destination_blob_pathname)
    zipbytes = io.BytesIO(blob.download_as_string())

    if is_zipfile(zipbytes):
        with ZipFile(zipbytes, 'r') as myzip:
            for contentfilename in myzip.namelist():
                contentfile = myzip.read(contentfilename)
                print("content file:", contentfilename)
                blob = bucket.blob(zipfilename_with_path.split(".")[0] + "/" + contentfilename)
                blob.upload_from_string(contentfile)
                
                
from gcsfs import GCSFileSystem
fs = GCSFileSystem()
fp = fs.open('gs://agriculture-bucket-wgr/Data sets/US-Wgr/2014-01-03_LE70460292014003EDC00_ETA/LE70460292014003EDC00_ETa.eta.tif')
with rio.open(fp) as src:
    im = src.read(1, masked=True)
    spatial_extent = plotting_extent(src)
    data = src.read(1)
    utm = pyproj.Proj(src.crs)
    lonlat = pyproj.Proj(init='epsg:4326')
    lon,lat = (-122.656026, 45.112865)
    east,north = pyproj.transform(lonlat, utm, lon, lat)
    metric_name = "ET"
    print(metric_name + '\n-------')
    print(f'Longitude,Latitude=\t\t({lon:.2f},{lat:.2f})')
    print(f'East,North=\t({east:g},{north:g})')

    row, col = src.index(east, north) # spatial --> image coordinates
    print(f'Row,Col=\t\t({row},{col})')
    val, neighbors_avg_val = get_neighbors(data, row, col)
    print("Average value of pixels:", neighbors_avg_val)

zipextract(BUCKET_NAME, "Data sets/US-Wgr/2014-01-03_LE70460292014003EDC00_ETA.zip")

content file: LE70460292014003EDC00_ETa.eta.tfw
content file: LE70460292014003EDC00_ETa.eta.tfw
content file: LE70460292014003EDC00_ETa.eta.tif
content file: LE70460292014003EDC00_ETa.eta.tif


In [32]:
from gcsfs import GCSFileSystem
fs = GCSFileSystem()
fp = fs.open('gs://agriculture-bucket-wgr/Data sets/US-Wgr/2014-01-03_LE70460292014003EDC00_ETA/LE70460292014003EDC00_ETa.eta.tif')
with rio.open(fp) as src:
    im = src.read(1, masked=True)
    spatial_extent = plotting_extent(src)
    data = src.read(1)
    utm = pyproj.Proj(src.crs)
    lonlat = pyproj.Proj(init='epsg:4326')
    lon,lat = (-122.656026, 45.112865)
    east,north = pyproj.transform(lonlat, utm, lon, lat)
    metric_name = "ET"
    print(metric_name + '\n-------')
    print(f'Longitude,Latitude=\t\t({lon:.2f},{lat:.2f})')
    print(f'East,North=\t({east:g},{north:g})')

    row, col = src.index(east, north) # spatial --> image coordinates
    print(f'Row,Col=\t\t({row},{col})')
    val, neighbors_avg_val = get_neighbors(data, row, col)
    print("Average value of pixels:", neighbors_avg_val)

ET
-------
Longitude,Latitude=		(-122.66,45.11)
East,North=	(527057,4.99555e+06)
Row,Col=		(1782,4739)
values of all pixels: [0.0006764627, 0.0007485954, 0.0007315389, 0.0, 0.0008389264, 0.0, 0.0006458672, 0.00079700345, 0.00074459333]
Average value of pixels: 0.0005758875
ET
-------
Longitude,Latitude=		(-122.66,45.11)
East,North=	(527057,4.99555e+06)
Row,Col=		(1782,4739)
values of all pixels: [0.0006764627, 0.0007485954, 0.0007315389, 0.0, 0.0008389264, 0.0, 0.0006458672, 0.00079700345, 0.00074459333]
Average value of pixels: 0.0005758875


In [ ]:

#             with open(file_path, "wb") as f:
#                 f.write(response.content)
#                 blob.upload_from_filename(file_path)
#                 unzip_all(site_path, file_name)
#                 print("please work")

    #         with open(blob_file_path, "wb") as f:
    #             print("in write")
    #             f.write(response.content)
    #             blob.upload_from_filename(file_path)
    #             unzip_all(site_path, file_name)
    #             print("please work")

    #             upload_files(BUCKET_NAME, site_path, file_name)
    #             fullPath = site_path + file_name
    #             unzip_all(site_path, file_name)